# Create a masked EnMAP image

This notebook reads an EnMAP Zarr dataset from the AVL data store, applies a cloud mask, optionally adjusts the co-ordinates, and saves the result locally as a TIFF file to use as an input for the subsequent processing notebooks.

In [1]:
import numpy as np
import xarray as xr
import rioxarray

List the available EnMAP Zarr datasets on AVL. You can also find this list
[on the Forum](https://forum.agriculturevlab.eu/t/enmap-data-access/46).

In [2]:
[data_id for data_id in data_store.list_data_ids() if data_id.startswith("enmap/")]

['enmap/Argentina/L2A-DT0000050274_20231103T143153Z_001_V010402_20240508T234610Z.zarr',
 'enmap/Argentina/L2A-DT0000050274_20231103T143158Z_002_V010402_20240508T234508Z.zarr',
 'enmap/Argentina/L2A-DT0000050274_20231103T143202Z_003_V010402_20240508T234256Z.zarr',
 'enmap/Argentina/L2A-DT0000052335_20231130T143048Z_002_V010402_20240808T213202Z.zarr',
 'enmap/Argentina/L2A-DT0000052335_20231130T143052Z_003_V010402_20240808T213031Z.zarr',
 'enmap/Argentina/L2A-DT0000052335_20231130T143057Z_004_V010402_20240508T233932Z.zarr',
 'enmap/Argentina/L2A-DT0000052335_20231130T143057Z_004_V010402_20240808T212142Z.zarr',
 'enmap/Argentina/L2A-DT0000052335_20231130T143101Z_005_V010402_20240808T212854Z.zarr',
 'enmap/Argentina/L2A-DT0000056394_20240104T143858Z_002_V010402_20240508T233315Z.zarr',
 'enmap/Argentina/L2A-DT0000056394_20240104T143858Z_002_V010402_20240808T205506Z.zarr',
 'enmap/Argentina/L2A-DT0000056394_20240104T143902Z_003_V010402_20240508T233350Z.zarr',
 'enmap/Argentina/L2A-DT00000563

Select a dataset identifier. If you need resampling, define the y extent of the global resolution. If you don't need resampling, you can set `y_max` and `y_min` to `None`.

In [3]:
input_zarr_id = "enmap/Belgium/L2A-DT0000013763_20230421T111150Z_003_V010201_20230428T144916Z.zarr"
y_max = 5627820
y_min = 5586300

Open the selected dataset.

In [4]:
enmap = data_store.open_data(input_zarr_id)

Mask the reflectance data, and apply the new y co-ordinates if they've been defined.

In [5]:
enmap_masked = (
    enmap.reflectance.where(
        (enmap.cloud == 0) &
        (enmap.cloudshadow == 0) &
        (enmap.haze == 0) & 
        (enmap.cirrus == 0)
    )
)

if None not in [y_max, y_min]:
    new_y_coords = np.arange(y_max, y_min, -30, dtype=np.float64)
    enmap_masked = enmap_masked.interp({"y": new_y_coords}, method="nearest")

enmap_masked = enmap_masked.astype(np.float32)

Save the image, applying LZW compression to reduce the file size. This may take about a minute to complete.

In [6]:
enmap_masked.rio.to_raster("images/EnMAP_resampled_notglob.tif", compress="LZW")